# Linear Models

In [6]:
import glob
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Get Data

In [32]:
def get_subject_from_path(path):
    normalized_path = os.path.normpath(path)
    path_components = normalized_path.split(os.sep)
    return path_components[-2][4:]

In [33]:
# Search for all functional connectivity files and read them into a
# numpy array.
fc_path = '/imaging3/owenlab/bpho/python_power_fc'
fc_paths = glob.glob(fc_path + '/**/power_fc.npy', recursive=True)

fcs = {}
for path in fc_paths:
    subject_id = get_subject_from_path(path)
    # print(subject_id)
    subject_fc = np.load(path)
    fcs[subject_id] = subject_fc[np.triu_indices(264, k=1)]
print(len(fcs))

722


In [35]:
print(fcs["NDARAP912JK3"].shape)

(34716,)


## Get Labels

In [38]:
wisc_label_path = "/imaging3/owenlab/bpho/Biobank Labels/Subjects with WISC.csv"
wisc_labels = pd.read_csv(wisc_label_path)
wisc_labels.set_index(keys='assessment WISC,EID', inplace=True)
display(wisc_labels)

,Unnamed: 0,"assessment WISC,Administration","assessment WISC,Comment_ID","assessment WISC,Data_entry","assessment WISC,Days_Baseline","assessment WISC,PSCID","assessment WISC,START_DATE","assessment WISC,Season","assessment WISC,Site","assessment WISC,Study",...,"assessment WISC,WISC_VSI_Percentile","assessment WISC,WISC_VSI_Sum","assessment WISC,WISC_Vocab_Raw","assessment WISC,WISC_Vocab_Scaled","assessment WISC,WISC_WMI","assessment WISC,WISC_WMI_Percentile","assessment WISC,WISC_WMI_Sum","assessment WISC,WISC_complete","assessment WISC,WISC_incomplete_reason","assessment WISC,Year"
"assessment WISC,EID",,,,,,,,,,,,,,,,,,,,,
NDARAC331VEH,0,All,NaN,Complete,2,NaN,1901-01-01T07:00:00Z,Spring,3,HBN,...,97,30,40,13,110,75.0,23,1,NaN,2019
NDARAC462DZH,1,All,NaN,Complete,74,NaN,1901-01-01T07:00:00Z,Spring,1,HBN,...,2,9,18,10,76,5.0,12,1,NaN,2019
NDARAF440XWG,2,All,NaN,Complete,104,NaN,1901-01-01T07:00:00Z,Spring,3,HBN,...,90,27,15,9,110,75.0,23,1,NaN,2019
NDARAF535XK6,3,All,NaN,Complete,77,NaN,1901-01-01T07:00:00Z,Spring,3,HBN,...,10,13,18,4,97,42.0,19,1,NaN,2019
NDARAG115LZP,4,All,NaN,Complete,135,NaN,1901-01-01T07:00:00Z,Spring,3,HBN,...,96,29,32,9,91,27.0,17,1,NaN,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NDARPL406KX4,432,All,NaN,Complete,522,NaN,1901-01-01T07:00:00Z,Fall,1,HBN,...,34,18,16,7,88,21.0,16,1,NaN,2018
NDARRL426AD5,433,All,NaN,Complete,119,NaN,1901-01-01T07:00:00Z,Fall,1,HBN,...,10,13,25,9,91,27.0,17,1,NaN,2018
NDARTT272WT5,434,All,NaN,Complete,385,NaN,1901-01-01T07:00:00Z,Summer,1,HBN,...,55,21,34,11,100,50.0,20,1,NaN,2019


In [42]:
label_subject_ids = wisc_labels.index
subjects_with_wisc = {}

for subject_id in label_subject_ids:
    if subject_id not in fcs:
        continue
    
    subject_wisc_fsiq_sum = wisc_labels.at[subject_id, 'assessment WISC,WISC_FSIQ_Sum']
    subjects_with_wisc[subject_id] = (fcs[subject_id], subject_wisc_fsiq_sum)

print(subjects_with_wisc['NDARAC331VEH'])

(array([ 0.58186466, -0.05859981,  0.08431777, ...,  0.39561737,
        0.3742862 ,  0.6218255 ], dtype=float32), 89)
